<a href="https://colab.research.google.com/github/MengOonLee/LLM/blob/main/References/LangChain/ipynb/T10_Local.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
%%bash

pip install --no-cache-dir -qU \
    torch torchvision lightning transformers accelerate \
    langchain langgraph langchain-core langchain-community \
    langchain-huggingface huggingface_hub

In [ ]:
!hf auth login

In [ ]:
import os
import warnings
warnings.filterwarnings("ignore")
import transformers
import time

model_name = "google/gemma-3-1b-it"
local_path = "./models"
os.makedirs(local_path, exist_ok=True)

start_time = time.time()
model = transformers.AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name)
model.save_pretrained(save_directory=local_path)
tokenizer = transformers.AutoTokenizer.from_pretrained(
    pretrained_model_name_or_path=model_name)
tokenizer.save_pretrained(save_directory=local_path)
end_time = time.time() - start_time

print("Time taken: %.2f"%(end_time))

In [ ]:
import transformers
from langchain_huggingface import llms

model = transformers.AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=local_path)
tokenizer = transformers.AutoTokenizer.from_pretrained(
    pretrained_model_name_or_path=local_path)

pipe = transformers.pipeline(model=model, tokenizer=tokenizer,
    task="text-generation")
llm = llms.HuggingFacePipeline(pipeline=pipe)

In [5]:
from langchain_core import prompts
import time

template = """
Question: {question}
"""
prompt = prompts.PromptTemplate.from_template(template)

chain = prompt | llm

question = "What is SWIFT financial messaging?"

start_time = time.time()
response = chain.invoke(input={"question": question})
end_time = time.time() - start_time

print("Time take: %.4f, %s"%(end_time, response))

Time take: 94.9742, 
Question: What is SWIFT financial messaging?
**Answer:** SWIFT (Society for Worldwide Interbank Financial Telecommunication) is a global financial messaging network that facilitates the transfer of funds between banks.
**Additional information:** SWIFT is a crucial component of the international payments system.
**Explanation:** SWIFT is a secure, private network that allows banks around the world to communicate about payments. It's used to send money, transfer accounts, and process transactions across borders.

Would you like me to provide more details about SWIFT?

